In [1]:
import numpy as np
from obspy import read_events
from obspy import read_inventory
from obspy.clients.fdsn import Client
client_wm = Client("IRIS")
from obspy.clients.iris import Client  
from obspy.taup import TauPyModel
from obspy import Stream, Trace
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from obspy import UTCDateTime
import matplotlib.pyplot as plt
from obspy.geodetics import locations2degrees,degrees2kilometers
from scipy import signal
plt.rc('font', size=10) 
%matplotlib widget
import datetime
from obspy.signal.trigger import recursive_sta_lta,plot_trigger,coincidence_trigger
import pandas as pd
from matplotlib.dates import date2num
from obspy.imaging.util import (_set_xaxis_obspy_dates, _id_key, _timestring)
from matplotlib.ticker import MaxNLocator, ScalarFormatter

In [4]:
evid_dum='https://earthquake.usgs.gov/fdsnws/event/1/query?&starttime=2021-09-13&endtime=2021-09-14&latitude=67&longitude=-145&maxradius=5'

In [5]:
cat_dum = read_events(evid_dum)


In [7]:
start_time=UTCDateTime(2020,9,1,0,0,0)
end_time=UTCDateTime(2020,9,1,12,0,0)
st_10day = client_wm.get_waveforms("AK", "SCRK", "*", "BHZ", start_time, end_time)


In [12]:
trc=st_10day[0].detrend().filter('bandpass', freqmin=0.5,freqmax=25)
#trc=st_10day[0].detrend()

/Users/yuantian/anaconda3/envs/obspy/lib/python3.7/site-packages/obspy/signal/filter.py:67: UserWarning: Selected high corner frequency (25) of bandpass is at or above Nyquist (25.0). Applying a high-pass instead.
  warnings.warn(msg)


In [75]:
SECONDS_PER_DAY=3600*24
trace=trc.slice(starttime=start_time+3000,endtime=start_time+4200)
x_values = ((trace.times() / SECONDS_PER_DAY) +date2num(trace.stats.starttime.datetime))
fig2 = plt.figure(figsize=(12,5))
df = trace.stats.sampling_rate
npts = trace.stats.npts
#t = np.arange(npts, dtype=np.float32) / df
ax = fig2.add_subplot(211)
_set_xaxis_obspy_dates(ax)
l3,=plt.plot(x_values,trace.data,label='displacement')
plt.setp(ax.get_xticklabels())
ax.text(0.02, 0.95, trace.id, transform=ax.transAxes,fontdict=dict(fontsize="small", ha='left', va='top'),bbox=dict(boxstyle="round", fc="w", alpha=0.8))
plt.setp(ax.get_yticklabels(), fontsize='small')
ax.yaxis.set_major_locator(MaxNLocator(7, prune="both"))
ax.yaxis.set_major_formatter(ScalarFormatter())

th_fold=10
window_len=50*60*2
record_time=30*60
record_number=record_time*trace.stats.sampling_rate
abs_st_data=np.abs(trace.data)
ind_th=[]
recording=False
rtt=0
t_stp=0
cm_sum=0
ltavg=np.sum(abs_st_data[0:window_len])/window_len
window_v=[]
cm_sum_s=[]
ur_a=[]
for i,abs_st in enumerate(abs_st_data):
    rtt=rtt+1
    t_stp=t_stp+1
    if rtt>window_len:
        rtt=0
        ltavg=cm_sum/window_len
        #print(ltavg)
        cm_sum=abs_st
        plt.plot([x_values[t_stp-1],x_values[t_stp-1]],[300,-200],'r--',label='window boundary')
    cm_sum=cm_sum+abs_st
    cm_sum_s.append(cm_sum/window_len*th_fold)
    window_v.append(th_fold*ltavg)
    ur_a.append(abs_st/ltavg)
    if abs_st>th_fold*ltavg and recording is False:
        ind_th.append(i)
        recording=True
    if ind_th!=[] and (i-ind_th[-1])>record_number:
        recording=False
l2,=plt.plot(x_values,window_v,'g-',label=r'$R*\overline{|u_{n}|}$  (R=10)')
l4,=plt.plot(x_values,cm_sum_s,'b-',label=r'running summation of $R*|u_{ti}|$')
#plt.legend(handles=[l2])

plt.legend(handles=[l3,l2,l4],loc=1)

ax = fig2.add_subplot(212)
_set_xaxis_obspy_dates(ax)
plt.plot(x_values,ur_a)
plt.setp(ax.get_xticklabels())
ax.text(0.02, 0.95, trace.id, transform=ax.transAxes,fontdict=dict(fontsize="small", ha='left', va='top'),bbox=dict(boxstyle="round", fc="w", alpha=0.8))
plt.setp(ax.get_yticklabels(), fontsize='small')
ax.yaxis.set_major_locator(MaxNLocator(7, prune="both"))
ax.yaxis.set_major_formatter(ScalarFormatter())
plt.plot(x_values,np.ones_like(x_values)*10,'g-')
plt.legend([r'displacement ratio $|u|/{\overline{|u_{n}|}}$','R'])

/Users/yuantian/anaconda3/envs/obspy/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  after removing the cwd from sys.path.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

  agg_filter: a filter function, which takes a (m, n, 3) float array and a dpi value, and returns a (m, n, 3) array
  alpha: float
  animated: bool
  backgroundcolor: color
  bbox: dict with properties for `.patches.FancyBboxPatch`
  clip_box: `.Bbox`
  clip_on: bool
  clip_path: [(`~matplotlib.path.Path`, `.Transform`) | `.Patch` | None]
  color or c: color
  contains: callable
  figure: `.Figure`
  fontfamily or family: {FONTNAME, 'serif', 'sans-serif', 'cursive', 'fantasy', 'monospace'}
  fontproperties or font_properties: `.font_manager.FontProperties`
  fontsize or size: {size in points, 'xx-small', 'x-small', 'small', 'medium', 'large', 'x-large', 'xx-large'}
  fontstretch or stretch: {a numeric value in range 0-1000, 'ultra-condensed', 'extra-condensed', 'condensed', 'semi-condensed', 'normal', 'semi-expanded', 'expanded', 'extra-expanded', 'ultra-expanded'}
  fontstyle or style: {'normal', 'italic', 'oblique'}
  fontvariant or variant: {'normal', 'small-caps'}
  fontweight or w